# Project3_v3 — Optimized Backtest (Phase1 + Phase2)

**新增功能**：
- Phase1：next‑bar 成交 + 风控规则 (A/B/D)
- Phase2：参数扫描 + 结果汇总

**目标**：速度、可复现、结构化模块化、策略准确性。

## 0) Setup & Reproducibility

In [ ]:
from __future__ import annotations

from pathlib import Path
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from project3.optimized_integration_v3 import (
    run_optimized_backtest_v3,
    plot_optimized_performance_v3,
    summarize_performance_v3,
)
from optimized_backtest_v3 import RiskConfig

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
print(f"Reproducibility seed set: {SEED}")

## 1) Configuration

In [ ]:
# Data selection
BASE_DIR = Path('.')
CANDIDATE_FILES = [
    BASE_DIR / 'real_akshare_spread_data.csv',
    BASE_DIR / 'crude_oil_wti_spread_data.csv',
    BASE_DIR / 'demo_spread_data.csv',
]

csv_path = next((p for p in CANDIDATE_FILES if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError(
        'No spread CSV found. Place a CSV with NEAR/FAR columns in this folder.'
    )

symbol = 'CALENDAR_SPREAD'
initial_capital = 500_000.0
quantity = 10
lookback_window = 30
z_threshold = 1.5
commission = 5.0
slippage = 0.01

# Phase1: execution delay + risk rules
execution_delay = 1  # next-bar fill
max_leverage = 2.0
max_position_per_leg = quantity * 2
max_drawdown_pct = 0.10

print(f'Using data file: {csv_path}')

## 2) Data Validation & Preview

In [ ]:
df = pd.read_csv(csv_path, index_col=0, parse_dates=True)
required_columns = {'NEAR', 'FAR'}
if not required_columns.issubset(df.columns):
    raise ValueError(f'Missing required columns: {required_columns - set(df.columns)}')

df = df.sort_index()
missing_count = df[['NEAR', 'FAR']].isna().sum().sum()
if missing_count > 0:
    print(f'Warning: {missing_count} missing values found in NEAR/FAR columns.')

print(df[['NEAR', 'FAR']].head())
print(f'Data shape: {df.shape}, Date range: {df.index.min()} -> {df.index.max()}')

## 3) Phase1: Risk Rules + Next‑Bar Execution

In [ ]:
def cash_sufficient(cash, est_cost):
    return cash >= est_cost

def leverage_ok(equity, gross_exposure, max_leverage):
    if equity <= 0:
        return False
    return gross_exposure <= equity * max_leverage

def drawdown_ok(equity_curve, max_dd):
    if equity_curve.empty:
        return True
    running_max = equity_curve['total'].cummax()
    dd = (equity_curve['total'] / running_max - 1.0).min()
    return dd >= -max_dd

## 4) Run v3 Backtest (Phase1 applied)

In [ ]:
# Run optimized backtest (Phase1 rules applied post‑run as validation gate)
risk_config = RiskConfig(
    max_leverage=max_leverage,
    max_position_per_leg=max_position_per_leg,
    max_drawdown_pct=max_drawdown_pct,
)

result = run_optimized_backtest_v3(
    csv_path=str(csv_path),
    symbol=symbol,
    initial_capital=initial_capital,
    quantity=quantity,
    lookback_window=lookback_window,
    z_threshold=z_threshold,
    commission=commission,
    slippage=slippage,
    execution_delay=execution_delay,
    risk_config=risk_config,
)

summarize_performance_v3(result)

# Phase1 checks (risk gates)
equity_curve = result.performance['equity_curve']
print('Drawdown OK:', drawdown_ok(equity_curve, max_drawdown_pct))

## 5) Phase2: Parameter Scan

In [ ]:
scan_lookback = [20, 30, 40, 60, 90, 120]
scan_z = [1.0, 1.5, 2.0, 2.5, 3.0]
scan_qty = [1, 5, 10, 20]
scan_commission = [2.5, 5.0, 7.5]
scan_slippage = [0.005, 0.01, 0.02]

rows = []
for lw in scan_lookback:
    for z in scan_z:
        for q in scan_qty:
            for c in scan_commission:
                for s in scan_slippage:
                    r = run_optimized_backtest_v3(
                        csv_path=str(csv_path),
                        symbol=symbol,
                        initial_capital=initial_capital,
                        quantity=q,
                        lookback_window=lw,
                        z_threshold=z,
                        commission=c,
                        slippage=s,
                        execution_delay=execution_delay,
                        risk_config=risk_config,
                    )
                    perf = r.performance
                    rows.append({
                        'lookback': lw,
                        'z_threshold': z,
                        'quantity': q,
                        'commission': c,
                        'slippage': s,
                        'total_return': perf['total_return'],
                        'sharpe': perf['sharpe_ratio'],
                        'max_drawdown': perf['max_drawdown'],
                        'volatility': perf['volatility'],
                    })

scan_df = pd.DataFrame(rows)
scan_df = scan_df.sort_values('sharpe', ascending=False)
display(scan_df.head(20))

## 6) Plots

In [ ]:
plot_optimized_performance_v3(
    result,
    lookback_window=lookback_window,
    z_threshold=z_threshold,
    title='Project3_v3 — Optimized Backtest with Phase1/2',
)

## 7) References

- IBKR: slippage in model backtesting  
  https://www.interactivebrokers.com/campus/ibkr-quant-news/slippage-in-model-backtesting/
- Backtrader slippage model  
  https://www.backtrader.com/docu/slippage/slippage/
- Commission + slippage handling example  
  https://www.slingacademy.com/article/handling-commission-and-slippage-in-backtrader/
- Execution delay slippage modeling  
  https://support.eareview.net/support/solutions/articles/19000052275-using-execution-delay-slippage